## Euclidean Distance, Model Implementation

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def euclidean_distance(stacked_tensor):
    reduced_tensor = stacked_tensor[:, :, 0, :] - stacked_tensor[:, :, 1, :]
    tensor_squared = reduced_tensor**2
    target = tensor_squared.sum()
    target = torch.sqrt(target)
    return target

class EuclideanModel(nn.Module):
    def __init__(self):
        super(EuclideanModel, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(2,1), bias = False)       
        self.fc = None

    def activation(self, x):
        return x**2
    
    def forward(self, x):

        conv_out = self.conv(x)    
        conv_out = self.activation(conv_out)
        conv_out_flat = conv_out.reshape(conv_out.size(0), -1)

        self.fc = None
        tensor_width = conv_out.reshape(conv_out.size(0), -1).size(1)
        self.fc = nn.Linear(tensor_width, 1, bias=False)

        with torch.no_grad():  # Disable gradient tracking for manual weight assignment
            self.fc.weight.fill_(1)
    
        output = self.fc(conv_out_flat)
        output = torch.sqrt(output)

        return output

### Training Loop

In [10]:
from tqdm import tqdm 
from torch.optim.lr_scheduler import StepLR, LambdaLR
from torch.utils.data import DataLoader, TensorDataset
torch.manual_seed(2)

criterion = nn.MSELoss()
trained_model = EuclideanModel()

# Generate random tensors to train
num_samples = 50
batch_size = 50
vector_dim = 15

mean = 2
std = 3.9
data = mean + std * torch.randn(num_samples, 2, 15)
data = data.unsqueeze(1)
data.shape

optimizer = optim.AdamW(trained_model.parameters(), lr=0.01)

# Construct a learning rate lambda
def lr_lambda(batch_idx):

    decay_interval = 2
    decay_rate = 0.9
    if batch_idx % decay_interval == 0 and batch_idx > 0:
        return decay_rate
    return 1

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

# Create data loader
dataset = TensorDataset(data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Number of epochs for training
epochs = 1000

# Training loop
for epoch in tqdm(range(epochs), desc = "Training Epoch: "):
    for batch_idx, (inputs,) in enumerate(dataloader):
        # Forward pass
        outputs = trained_model(inputs)
        
        # Assuming the targets are the same as the inputs (for simplicity)
        targets = torch.stack([euclidean_distance(inputs[i:i+1]) for i in range(inputs.size(0))]).view(batch_size, 1)
        
        # Compute loss
        loss = criterion(outputs, targets)*10
        
        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()        # Backpropagate the loss
        optimizer.step()       # Update model parameters
        scheduler.step()
        
print(f"These are the weights: {trained_model.conv.weight}")
print(f"THIS is the lOSS: {loss}")

Training Epoch: 100%|██████████| 1000/1000 [00:04<00:00, 248.05it/s]

These are the weights: Parameter containing:
tensor([[[[ 0.9990],
          [-0.9987]]]], requires_grad=True)
THIS is the lOSS: 0.006159065291285515


### Model Assessment

In [11]:
# Test performance against a random tensor 
dim = 15
input_tensor = torch.cat((torch.randn(1, 1, 1, dim), torch.randn(1, 1, 1, dim)), dim=2)

error = abs(trained_model(input_tensor) - euclidean_distance(input_tensor)).cpu().detach().numpy()[0][0]
print(f"The error relative to true value is {error}")

The error relative to true value is 0.007527828216552734
